In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import itertools
from scipy.spatial import cKDTree
import geopy.distance
from operator import itemgetter

In [2]:
def ckdnearest(brick_kilns, rivers, gdfB_cols=['geometry']):
        A = np.vstack([np.array(geom) for geom in brick_kilns[['lon', 'lat']].values])
        B = [np.array(geom.coords) for geom in rivers.geometry.to_list()]
        B_ix = tuple(itertools.chain.from_iterable(
            [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]))
        B = np.concatenate(B)
        ckd_tree = cKDTree(B)
        dist, river_point_idx = ckd_tree.query(A, k=1)
        closest_river_point = B[river_point_idx]
        river_origin_idx = itemgetter(*river_point_idx)(B_ix)
        gdf = pd.concat(
            [brick_kilns, rivers.loc[river_origin_idx, gdfB_cols].reset_index(drop=True),
             pd.DataFrame(closest_river_point, columns=['closest_river_point_long', 'closest_river_point_lat']),
             pd.Series(dist, name='dist')], axis=1)
        return gdf


def cal_bk_river_dist(path, waterways):
        state_bk = pd.read_csv(path)
        bk_river_mapping = ckdnearest(state_bk, waterways)
        bk_river_mapping['distance'] = 0
        for i in range(len(state_bk)):
            bk_river_mapping['distance'][i] = geopy.distance.distance(
                (bk_river_mapping['lat'][i], bk_river_mapping['lon'][i]),
                (bk_river_mapping['closest_river_point_lat'][i], bk_river_mapping['closest_river_point_long'][i])
            ).km
        return bk_river_mapping

In [18]:
waterways_path = '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/waterways.shp'
waterways = gpd.read_file(waterways_path)

# Load brick kilns data
brick_kilns_paths = {
    'All' : '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/combined_file.csv',
    "Punjab": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/punjab.csv',
    "Haryana": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/haryana.csv',
    "Bihar": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/bihar.csv',
    "Uttar Pradesh": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/uttar_pradesh.csv',
    "West Bengal": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/west_bengal.csv',
}
state = 'West Bengal'
brick_kilns_path = brick_kilns_paths[state]

bk_river_mapping = cal_bk_river_dist(brick_kilns_path, waterways)
bk_river_mapping.to_csv(f'./{state}_bk_river_distance.csv')

/tmp/ipykernel_203670/3177182188.py:23: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  bk_river_mapping['distance'][i] = geopy.distance.distance(
/tmp/ipykernel_203670/3177182188.py:23: SettingWithCopyWarning: 
A value is trying to be set on 

In [19]:
bk_river_mapping.head(5)

,parent,line_id,class,lat,lon,state,x_c,y_c,w,h,...,bottom_right_y,local_top_left_x,local_top_left_y,local_bottom_right_x,local_bottom_right_y,geometry,closest_river_point_long,closest_river_point_lat,dist,distance
0,"22.25,88.55",0,zigzag,22.249829,88.554479,west_bengal,2.503109e+07,1.464920e+07,0.044787,0.069990,...,1.464924e+07,952.436128,538.016808,1002.597792,616.405272,"LINESTRING (88.51401 22.34519, 88.51510 22.345...",88.604777,22.300813,0.071618,7.664456
1,"23.73,87.71",0,zigzag,23.732203,87.715413,west_bengal,2.495289e+07,1.449911e+07,0.065385,0.042536,...,1.449913e+07,1027.919648,311.870832,1101.151072,359.510928,"LINESTRING (87.69393 23.71135, 87.69458 23.711...",87.724803,23.722809,0.013283,1.413985
2,"23.55,87.90",0,zigzag,23.550058,87.904966,west_bengal,2.497055e+07,1.451764e+07,0.056742,0.025881,...,1.451765e+07,991.112528,539.566216,1054.663792,568.552824,"LINESTRING (87.90581 23.54237, 87.91011 23.549...",87.910109,23.549142,0.005224,0.534798
3,"22.53,88.96",0,zigzag,22.527043,88.965536,west_bengal,2.506941e+07,1.462126e+07,0.078170,0.055008,...,1.462129e+07,1032.251640,827.612856,1119.801480,889.221704,"LINESTRING (88.84887 22.70603, 88.84887 22.702...",88.967754,22.530673,0.004254,0.462254
4,"22.62,88.56",0,zigzag,22.622992,88.558923,west_bengal,2.503151e+07,1.461157e+07,0.062495,0.026909,...,1.461159e+07,424.625208,242.853240,494.619272,272.991880,"LINESTRING (88.42659 22.79812, 88.42618 22.794...",88.558395,22.624519,0.001617,0.177699
